In [1]:
import sys
sys.path.append('/home/mila/l/leo.gagnon/latent_control')

%load_ext autoreload
%autoreload 2
import torch
import pyvene as pv
import pprint
from task import MetaLearningTask, FineTuningTask
from transformers import BatchEncoding, PretrainedConfig
import jax.random as jr
from torch2jax import j2t, t2j
import dataclasses
import pyreft
import nvidia_smi
from models.mamba import MambaLMHeadModel
from dataclasses import dataclass
from mamba_ssm.utils.generation import InferenceParams
from task import DynamicIntervenableModel, TuneConfig, LoraConfig, ReftConfig
import gc
from models.gpt import GPT, GPTConfig
from models.mamba import MambaLMHeadModel, MambaConfig
from pympler import asizeof
from data.hmm import CompositionalHMMDataset, CompositionalHMMDatasetConfig
import lightning as L

/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/mamba_ssm/ops/selective_scan_interface.py:164: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, xz, conv1d_weight, conv1d_bias, x_proj_weight, delta_proj_weight,
/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/mamba_ssm/ops/selective_scan_interface.py:240: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, dout):
/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/mamba_ssm/ops/triton/layer_norm.py:986: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(
/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/mamba_ssm/ops/triton/layer_nor

In [2]:
#gpt : 9qs3bntc
#mamba: e72ebe43

In [1]:
a = [1,2,3,4]

In [3]:
a[-1::-1]

[4, 3, 2, 1]

In [27]:
task = FineTuningTask(
    TuneConfig(
        "9qs3bntc",
        ReftConfig(
            low_rank_dimension=2, layers=[3], timesteps=[-1], component="output"
        ),
        [[1, 0], [2, 1]],
        batch_size=32
    )
)
task.model.to("cuda");

number of parameters: 4.75M
Loaded dataset : (11288/1000)
Loaded checkpoing : epoch=894,step=40250.ckpt


In [3]:
trainer = L.Trainer(
    logger=None,
    max_steps=100,
    accelerator="gpu",
    enable_checkpointing=False,
    val_check_interval=10,
    reload_dataloaders_every_n_epochs=1,
    check_val_every_n_epoch=None,
)

/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/mila/l/leo.gagnon/latent_control/venv/lib/pyth ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [137]:
task.cfg.batch_size

256

In [4]:
trainer.fit(task)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type                     | Params | Mode 
------------------------------------------------------------------
0 | model        | DynamicIntervenableModel | 1.0 K  | train
  | other params | n/a                      | 4.8 M  | n/a  
------------------------------------------------------------------
0         Trainable params
4.8 M     Non-trainable params
4.8 M     Total params
19.316    Total estimated model params size (MB)
1         Modules in train mode
86        Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py:78: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 256. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (11) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 88.00 MiB. GPU 0 has a total capacity of 31.73 GiB of which 16.44 MiB is free. Including non-PyTorch memory, this process has 31.71 GiB memory in use. Of the allocated memory 29.62 GiB is allocated by PyTorch, and 1.68 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [42]:
import numpy as np

In [109]:
a=[0,1,2,3,4]

In [ ]:
slice()

In [111]:
a[-1:None:-1]

[4, 3, 2, 1, 0]

In [71]:
slice(*(-1,-2,-1))

slice(-1, -2, -1)

In [66]:
a[slice(-1,-2,-1)]

[5]

In [53]:
np.arange(10)[eval(':2')]

SyntaxError: invalid syntax (<string>, line 1)

In [41]:
slice(3,0,-1)

slice(3, 0, -1)

In [ ]:
torch.slice()

In [40]:
torch.arange(10).slice_scatter()

ValueError: step must be greater than zero

In [18]:
seqs = task.full_data.__getitems__(torch.arange(32), length=100)[0]

In [30]:
logits = task.model(seqs)

In [31]:
import nvidia_smi

nvidia_smi.nvmlInit()

handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
# card id 0 hardcoded here, there is also a call to get all available card ids, so we could iterate

info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)

print("Total memory:", info.total)
print("Free memory:", info.free)
print("Used memory:", info.used)

nvidia_smi.nvmlShutdown()


Total memory: 34359738368
Free memory: 29704519680
Used memory: 4655218688


In [28]:
del logits
gc.collect()
torch.cuda.empty_cache()

0

In [132]:
del logits
gc.collect()
torch.cuda.empty_cache()

In [18]:
torch.cuda.empty_cache()